## HTML — отдельно, Python — отдельно

Решение проблемы есть: HTML-код выносят в отдельные файлы (их называют **HTML-шаблонами**).

```html
<!-- homepage/index.html -->
<!DOCTYPE html>
<html lang="ru">
  <head>
    <title>ACME</title>
  </head>
  <body>
    <h1>Главная страница</h1>
  </body>
</html>
```

Чтобы в ответе на запрос Django мог вернуть HTML-код, шаблоны подключают через загрузчик `loader`.

```py
from django.http import HttpResponse
# Импортируем загрузчик.
from django.template import loader

def index(request):
    # Подключаем HTML-файл.
    template = loader.get_template('homepage/index.html')
    # Передаём в объект HttpResponse 
    # HTML-код из загруженного файла, объект запроса request;
    # и возвращаем этот объект.
    return HttpResponse(template.render({}, request))
```

В результате в ответ на запрос к главной странице вернётся HTML-код из файла *homepage/index.html*.

***
## Пиши, сокращай

Для работы с шаблонами разработчики Django придумали сокращённый синтаксис: view-функция возвращает функцию `render()`, в которую передаётся объект запроса `request` и адрес HTML-шаблона. 

Загрузка шаблона и создание объекта HttpResponse убраны «под капот», функция `render()` сама выполнит все эти операции.

```py
# Импортируем функцию render()
from django.shortcuts import render

def index(request):
    template_name = 'homepage/index.html'
    return render(request, template_name)
```

***
## Хранение шаблонов на уровне приложения

**В директориях приложений** создают папки *templates/имя_приложения/* и там хранят шаблоны, используемые в приложении. Такой порядок хранения шаблонов называют «на уровне приложения». 

```
...
├── acme_project/                  <-- Папка с настройками проекта
├── catalog/                       <-- Папка приложения catalog
│   ├── templates/                 <-- Директория для шаблонов
│   │   └── catalog/                 <-- Директория шаблонов приложения catalog
│   │       ├── product_detail.html  <-- Шаблон для отображения товара
│   │       └── product_list.html    <-- Шаблон для отображения списка товаров
│   ├── __init__.py
│   ├── admin.py   
│   ├── apps.py   
│   ├── models.py   
│   ├── test.py
│   ├── urls.py
│   └── views.py
├── homepage/                     <-- Папка приложения homepage
│   ├── templates/                <-- Директория для шаблонов
│   │   └── homepage/             <-- Директория шаблонов приложения homepage
│   │       └── index.html        <-- Шаблон главной страницы
│   └── ...                       <-- admin.py apps.py urls.py views.py и прочее
└── manage.py
```

***
## Хранение шаблонов на уровне проекта

При такой системе в корневой директории проекта создаётся папка *templates/*, в ней — директории, названные по именам приложений, и уже в них хранятся шаблоны, относящиеся к приложениям проекта. Такой порядок хранения шаблонов называют «на уровне проекта». 

Согласно этому подходу шаблоны приложения **catalog** должны лежать в директории *templates/catalog/*:

```
...
├── acme_project/             <-- Папка с настройками проекта
├── catalog/                  <-- Папка приложения catalog
├── homepage/                 <-- Папка приложения homepage
├── templates/                <-- Отдельная директория для шаблонов
│   ├── catalog/                  <-- Директория шаблонов приложения catalog
│   │   ├── product_detail.html   <-- Шаблон для отображения товара
│   │   └── product_list.html     <-- Шаблон для отображения списка товаров
│   └── homepage/                 <-- Директория шаблонов приложения homepage
│       └── index.html            <-- Шаблон для главной страницы
└── manage.py
```

***
## Настройки шаблонизатора Django

Разработчик должен указать фреймворку, где следует искать шаблоны и как с ними работать. Все эти настройки хранятся в списке `TEMPLATES` в файле `settings.py`.

* `BACKEND`: под этим ключом указывается, какой шаблонизатор используется в проекте.
Самые популярные шаблонизаторы, использующиеся в Django-проектах, — это `Jinja2` и Django Template Language (DTL).
По умолчанию в Django подключён DjangoTemplates. С ним мы и работаем.

* `DIRS`: здесь указывается список директорий, где Django будет искать шаблоны. При развёртывании Django-проекта создаётся константа `BASE_DIR`, в которой хранится адрес рабочей директории. Чтобы задать адрес директории с шаблонами, добавим к пути `BASE_DIR` вложенную папку `templates`.

* `APP_DIRS` показывает, нужно ли искать шаблоны в папках приложений.

Вот список `TEMPLATES` с настройками хранения шаблонов на уровне проекта:

```py
# acme_project/settings.py
# Адреса папок с шаблонами можно вынести в отдельные константы,
# код станет более читабельным.
TEMPLATES_DIR = BASE_DIR / 'templates'

TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        # Указываем, в каких директориях искать HTML-шаблоны.
        'DIRS': [TEMPLATES_DIR],

        # Оставляем True: шаблоны приложений будут искаться 
        # не только на уровне проекта, но и в директориях приложений.
        # Это необходимо для работы 
        # встроенных приложений (например, админки).
        'APP_DIRS': True, 
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ]
        },
    }
]
```

[Шпоргалка по оформлению html кода](https://code.s3.yandex.net/Python-dev/cheatsheets/025-django-pravila-oformlenija-html-i-shablonov-shpora/025-django-pravila-oformlenija-html-i-shablonov-shpora.html)